
## ORM: One-to-Many Relationships with SQLAlchemy

https://docs.sqlalchemy.org/en/20/orm/quickstart.html

Watch Video Tutorial

https://github.com/jod35/OnetoMany-SQLALCHEMY2.0/blob/main/README.md

In [1]:
from sqlalchemy.orm import DeclarativeBase, Mapped, mapped_column, relationship
from sqlalchemy import ForeignKey

"""
table programs
    - id int pk
    - name str
    - years_of_study

table courses
    - id int pk
    - title
    - code
    - program_id => fk => programs.id
"""

'\ntable programs\n    - id int pk\n    - name str\n    - years_of_study\n\ntable courses\n    - id int pk\n    - title\n    - code\n    - program_id => fk => programs.id\n'

Create Model